In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns

import os

In [56]:
os.path.dirname(os.getcwd())

'/Users/markus/Documents/Productivity/SETI/m3-lunar-project'

In [94]:
parent = os.path.dirname(os.getcwd())
input_files_path = os.path.join(parent, 'input files', 'su25')

input_path = None
for i in os.listdir(input_files_path):
    if i[-4:] == '.csv' and i[0] == 'c':
        input_path = os.path.join(input_files_path, i)
        print(i)
input_path

clean M3 wavelengths targeted global.csv


'/Users/markus/Documents/Productivity/SETI/m3-lunar-project/input files/su25/clean M3 wavelengths targeted global.csv'

In [95]:
df = pd.read_csv(input_path)

In [255]:
df.head()

,Wavelength (µm) M3 targeted images,"average every 4 channels, step n, sort","average every 4 channels, step n, sort, round 0.0001",Wavelength (µm) M3 global images
0,0.44602,0.460990,0.4610,0.4610
1,0.45600,0.500920,0.5009,0.5009
2,0.46598,0.540840,0.5408,0.5408
3,0.47596,0.580765,0.5808,0.5808
4,0.48595,0.620690,0.6207,0.6207


In [378]:
# change step size if past a breakpoint
# current wl: step to transition to
breakpoints = {
    "wl" :   [   0, 0.44, 0.68, 0.71, 1.53, 1.56, 1.60],#, 1.64, 9999],
    "step" : [   0,    4,    4,    3,    2,    3,     4]#,   5,    4]
}
wavelengths = np.array(df[df.columns[0]])
empirical = np.array(df[df.columns[-1]].dropna())

In [401]:
# average 4 at a time
# set step size to last breakpoint
# increment by step size
# round to 4 decimals

print(f"     wl: step change, index change")

i = 0
bp_idx = 0
output = []
while i+3 < len(wavelengths):
    # average over next 4
    avg = np.round(np.mean(wavelengths[i:i+4]),4)

    # if next step exists
    # and current wl > bp_idx wl
    while bp_idx + 1 < len(breakpoints["step"]) and wavelengths[i] > breakpoints["wl"][bp_idx]:
        bp_idx += 1
        print(f"{wavelengths[i]:0.5f}: step {breakpoints['step'][bp_idx-1]}->{breakpoints['step'][bp_idx]}, idx {bp_idx-1}->{bp_idx}")

    # save and increment by values consumed
    output.append(avg)
    i += breakpoints["step"][bp_idx]
output = np.array(output)

print(f"\nlengths:\n" + f"output    {len(output)}\n" + f"empirical {len(empirical)}\n")
print(f"sum diff:  {sum(output - empirical):f}")

print(f"\n" + f"idx: generated vs empirical")
for i in range(min(len(output), len(empirical))):
    if empirical[i] != output[i]:
        print(f"{i}: {output[i]:.04f} | {empirical[i]:.04f}")

     wl: step change, index change
0.44602: step 0->4, idx 0->1
0.44602: step 4->4, idx 1->2
0.68556: step 4->3, idx 2->3
0.71551: step 3->2, idx 3->4
1.53400: step 2->3, idx 4->5
1.56390: step 3->4, idx 5->6

lengths:
output    85
empirical 85

sum diff:  0.000100

idx: generated vs empirical
21: 1.0099 | 1.0100
61: 2.0580 | 2.0579
69: 2.3774 | 2.3773
78: 2.7366 | 2.7367
82: 2.8964 | 2.8963


In [355]:
### inconsistencies
## round up, not truncating
# mean(0.99497, 1.005, 1.0149, 1.0249) = 1.0099425 -> 1.0099, manual is 1.01 > 1.0099
## round down
# mean(2.3624, 2.3724, 2.3823, 2.3923) = 2.37735 -> 2.3774, manual is 2.3773 < 2.3774